In [1]:
import numpy as np
import pandas as pd
import re
from pandas import DataFrame
from sqlalchemy import create_engine
from matplotlib import pyplot as plt
import matplotlib as mpl

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn import cross_validation
from sqlalchemy import create_engine
%matplotlib inline
pd.set_option('display.max_colwidth', -1)

In [2]:
def get_information(query, host_db="cal-patent-lab.chhaitskv8dz.us-west-2.rds.amazonaws.com", 
                    username="***REMOVED***", password="***REMOVED***", db="***REMOVED***"):
    # Connecting to the fung db
    engine_fung = create_engine("mysql://{}:{}@{}/{}".format(
        username, password, host_db, db))
    connection_fung = engine_fung.connect()
    patent_info = pd.read_sql(
        query, engine_fung)
    connection_fung.close()
    return patent_info

# Predicting invalidation

First, we pull down all of the patents that have ever been brought before the PTAB

In [3]:
# Connecting to the db
host_db = "cal-patent-lab.chhaitskv8dz.us-west-2.rds.amazonaws.com"
username = "***REMOVED***"
password = "***REMOVED***"
db = "***REMOVED***"

engine = create_engine("mysql://{}:{}@{}/{}".format(
    username, password, host_db, db))
connection = engine.connect()
q = "SELECT case_id, patent_id, invalidated, denied, filing_date, decision_date, patent_owner, petitioner_name FROM ptab_cases"
ptab_patents = pd.read_sql(q, engine)
connection.close()

ptab_patents[:20]

,case_id,patent_id,invalidated,denied,filing_date,decision_date,patent_owner,petitioner_name
0,CBM2012-00001,6553350,1.0,0.0,2012-09-16,2013-01-09,"Versata Software, Inc","SAP America, Inc."
1,CBM2012-00002,6064970,1.0,0.0,2012-09-16,2013-01-25,Progressive Casualty Insurance Company,Liberty Mutual Insurance Company
2,CBM2012-00003,8140358,1.0,0.0,2012-09-16,2013-02-12,Progressive Casualty Insurance Company,Liberty Mutual Insurance Company
3,CBM2012-00004,6064970,1.0,0.0,2012-09-16,2013-01-25,Progressive Casualty Insurance Company,Liberty Mutual Insurance Company
4,CBM2012-00005,6675151,1.0,0.0,2012-09-21,2013-01-23,"Frontline Technologies, Inc.","CRS Advanced Technology, Inc ."
5,CBM2012-00007,5361201,1.0,0.0,2012-09-19,2013-01-31,"CoreLogic Solutions, LLC","INTERTHINX, INC."
6,CBM2012-00010,7124088,1.0,0.0,2012-09-29,2013-02-25,Progressive Casualty Insurance Company,Liberty Mutual Insurance Company
7,CBM2012-00011,7124088,0.0,1.0,2012-09-29,2013-02-25,Progressive Casualty Insurance Company,Liberty Mutual Insurance Company
8,CBM2013-00001,7877269,0.0,1.0,2012-10-03,2013-02-27,Progressive Casualty Insurance Company,Liberty Mutual Insurance Company
9,CBM2013-00002,7877269,1.0,0.0,2012-10-02,2013-02-27,Progressive Casualty Insurance Company,Liberty Mutual Insurance Company


In [4]:
# Next, we want to grab the claims text associated with each patent and insert it into the dataframe

host_db = "rosencrantz.berkeley.edu"
username = "uspto"
password = "***REMOVED***"
db = "uspto"

engine = create_engine("mysql://{}:{}@{}/{}".format(
    username, password, host_db, db))
connection = engine.connect()

patent_id = int(ptab_patents.patent_id[:1])

In [4]:
# Inserting concatenated claim text into ptab_patents

found = list()
for idx, patent_id in enumerate(ptab_patents.patent_id):
    q = "SELECT text from uspto.claim where patent_id = '{}'".format(patent_id)
    claims = pd.read_sql(q, engine)
    claims_concat = claims.text.str.cat()
    ptab_patents.loc[ptab_patents.patent_id == '{}'.format(patent_id), 'claim_text'] = claims_concat
    found.append(not claims.empty)
    if idx % 100 == 0:
        print("Processing patent {}".format(idx))

ProgrammingError: (_mysql_exceptions.ProgrammingError) (1146, "Table 'uspto.claim' doesn't exist") [SQL: "SELECT text from uspto.claim where patent_id = '6553350'"]

In [ ]:
# ptab_patents.to_pickle('ptab_patents.pkl')

Loading pickled data, deduplicating patents, and running model

In [5]:
ptab_patents = pd.read_pickle('ptab_patents.pkl')

In [29]:
# De-duplicating patents
df = ptab_patents.dropna(axis=0, how='any')
mask = (ptab_patents.claim_text.str.len() > 1)
df = df.loc[mask]
dups = df.duplicated(subset='claim_text', keep='first')
df = df.loc[~dups]


X = df.claim_text.as_matrix()
y = df.invalidated.as_matrix()

In [30]:
# merigng the claims and the metadata
all_cases = get_information("SELECT * FROM ptab_cases")
all_info = get_information('SELECT * FROM patent_info')
ptab_cases = all_cases[all_cases.denied.notnull()]
ptab_stats = ptab_cases.merge(all_info,suffixes = ["_ptab", "_patent"], on="patent_id")
final = df.merge(ptab_stats, suffixes = ["_algo", "_all"], on="patent_id")

In [36]:
# De-duplicating patents
df = final.dropna(axis=0, how='any')
mask = (final.claim_text.str.len() > 1)
df = df.loc[mask]
dups = df.duplicated(subset='claim_text', keep='first')
df = df.loc[~dups]
X = df.claim_text.as_matrix()
meta_X = pd.get_dummies(df[['petitioner_name', 'art_unit', 'examiner_name']]).as_matrix()

y = df.invalidated_all.as_matrix()

In [37]:
def stem(X):
    stemmer2 = SnowballStemmer('english')
    stemmer = PorterStemmer()
    stemmed_X = []

    for elem in X:
        words = elem.split()
        new_claim = [stemmer.stem(word.decode('utf-8')) for word in words]
        stemmed_X.append(' '.join(new_claim))
    return np.array(stemmed_X)

new_X = stem(X) 

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def train_model(X, meta, y, C):
    print(X.shape)
    print(meta.shape)
    data = np.hstack((np.asmatrix(X).T, meta))
    # data = meta
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(
        X, y, test_size=0.2, random_state=20)
    meta_train, meta_test, y_meta_train, y_meta_test = cross_validation.train_test_split(
        meta, y, test_size=0.2, random_state=20)
    
    tfidf = TfidfVectorizer()
    tfidf.fit(X)
    X_train = tfidf.transform(X_train)
    X_test = tfidf.transform(X_test)

    meta_X_train = np.hstack((X_train.toarray(), meta_train[:,1:]))
    meta_X_test = np.hstack((X_test.toarray(), meta_test[:,1:]))
    print(meta_X_train.shape)
    # svc_class = LinearSVC(C=C)
    # model = svc_class.fit(meta_X_train, y_train)
    # forest = RandomForestClassifier(n_estimators=10, min_samples_split=2, min_samples_leaf=2)
    # model = forest.fit(meta_X_train, y_train)
    logistic_class = LogisticRegression(dual=True)
    model = logistic_class.fit(meta_X_train, y_train)

    precision = precision_score(y_test, model.predict(meta_X_test))
    recall = recall_score(y_test, model.predict(meta_X_test))

    print("Training accuracy is {0}".format(model.score(meta_X_train, y_train)))
    print("Testing accuracy is {0}".format(model.score(meta_X_test, y_test)))
    print("Precision is {0}".format(precision))
    print("Recall is {0}".format(recall))

    vals = df.invalidated_all.value_counts()
    print("Outcome of guessing is {}".format(float(vals[0])/float(sum(vals))))
    print("N is {0}".format(sum(vals)))
    
    return model, X_test, y_test, tfidf

In [39]:
model, X_test, y_test, tf = train_model(X, meta_X, y, 0.83)

(2430L,)
(2430L, 2205L)
(1944L, 16437L)
Training accuracy is 0.929012345679
Testing accuracy is 0.703703703704
Precision is 0.668874172185
Recall is 0.517948717949
Outcome of guessing is 0.610288065844
N is 2430


Recall is high when we keep our false negatives low and precision is high when we keep our false positives low. Basically, this classifier is overindexing on precision relative to recall. A next step here would be to look at tradeoffs between precision and recall between different probabalistic classifiers. We probably also want to play with regularization.

In [ ]:
# only works when with random forest
np.argsort(model.feature_importances_)[-10:]
for feature in np.argsort(model.feature_importances_)[-10:]:
    for word in tf.vocabulary_:
        if tf.vocabulary_[word] == feature:
            print word